<a href="https://colab.research.google.com/github/irwanmul/Text_to_SQL/blob/main/OpenRouter_with_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# Run this Python code to fix metadata
from IPython.display import display, Javascript
display(Javascript('''
  google.colab.kernel.proxyPort(0, {'cache': false}).then(() => {
    const notebook = google.colab.global.notebook;
    notebook.metadata.widgets = {"state": {}, "version": "1.0"};
    notebook.save();
  });
'''))

<IPython.core.display.Javascript object>

In [2]:
!pip install faiss-cpu openai tiktoken sentence-transformers
!pip install -U langchain langchain-huggingface
!pip install -U langchain-community

In [10]:
!pip install faiss-cpu

In [11]:
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.schema import Document
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import requests
import yaml

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
cd drive/'My Drive/Colab Notebooks'

/content/drive/My Drive/Colab Notebooks


In [14]:
# === Load YAML schema ===
# revision
with open("schema_table.yaml", "r") as file:
    data = yaml.safe_load(file)

docs = []

# Create document for transaction metadata
metadata = data["transaction_metadata"]
metadata_content = (
    f"Transaction metadata:\n"
    f"- Name: {metadata['name']}\n"
    f"- Default currency: {metadata['currency_default']}\n"
    f"- Version: {metadata['version']}\n"
    f"- Last updated: {metadata['last_updated']}"
)
docs.append(Document(page_content=metadata_content))


# Create document for transactions structure
transaction_fields = [
    "id (integer): Unique transaction identifier",
    "date (string): Transaction date in YYYY-MM-DD format",
    "description (string): Description of the transaction",
    "amount (float): Transaction amount",
    "currency (string): Currency code (e.g., USD)",
    "category (string): Transaction category",
    "account (string): Account used for transaction",
    "status (string): Transaction status (completed/pending)"
]

transaction_schema_content = (
    "Transaction table structure:\n" +
    "\n".join([f"- {field}" for field in transaction_fields])
)
docs.append(Document(page_content=transaction_schema_content))

In [15]:
print(docs)

[Document(metadata={}, page_content='Transaction metadata:\n- Name: transaction\n- Default currency: USD\n- Version: 1.0\n- Last updated: 2023-10-18T14:30:00Z'), Document(metadata={}, page_content='Transaction table structure:\n- id (integer): Unique transaction identifier\n- date (string): Transaction date in YYYY-MM-DD format\n- description (string): Description of the transaction\n- amount (float): Transaction amount\n- currency (string): Currency code (e.g., USD)\n- category (string): Transaction category\n- account (string): Account used for transaction\n- status (string): Transaction status (completed/pending)')]


In [16]:
# === Embedding + FAISS Index ===
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
db = FAISS.from_documents(docs, embedding_model)

retriever = db.as_retriever(search_kwargs={"k": 3})

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
# === Prompt Template ===
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are a data assistant. Create a BigQuery SQL query based on the following context and questions. Only provide the SQL query without explanation. Do not include the “``sql” tag like the following

Context:
{context}

Question:
{question}

SQL:
""")

In [26]:
# === Custom LLM wrapper buat OpenRouter ===
from langchain.llms.base import LLM
from typing import List, Optional
from pydantic import Field

class OpenRouterLLM(LLM):
    api_key: str = Field(...)
    model: str = Field(...)
    temperature: float = 0.2

    @property
    def _llm_type(self) -> str:
        return "openrouter"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        payload = {
            "model": self.model,
            "messages": [
                {"role": "user", "content": prompt}
            ],
            "temperature": self.temperature,
        }
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json",
            "HTTP-Referer": "https://colab.research.google.com/",  # Required by OpenRouter
            "X-Title": "Your App Name"                  # Required by OpenRouter
        }

        response = requests.post("https://openrouter.ai/api/v1/chat/completions", headers=headers, json=payload)
        if response.status_code == 200:
            return response.json()["choices"][0]["message"]["content"].strip()
        else:
            raise Exception(f"Request failed: {response.status_code}, {response.text}")

In [30]:
# === Init LLM
llm = OpenRouterLLM(api_key="", model="microsoft/mai-ds-r1:free")

# === LangChain pipeline
from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt_template}
)

In [31]:
# === Run
question = "Calculate how many unique transaction"
result = chain.invoke(question)

print("=== SQL Output ===")
print(result['result'])


=== SQL Output ===
SELECT COUNT(DISTINCT id) FROM transaction;


In [32]:
# === Run
question = "Find the total revenue generated in 2023"
result = chain.invoke(question)

print("=== SQL Output ===")
print(result['result'])


=== SQL Output ===
SELECT SUM(amount) AS total_revenue
FROM transaction
WHERE status = 'completed'
AND date BETWEEN '2023-01-01' AND '2023-12-31';


In [33]:
# === Run
question = "Find the top 5 categories by total transaction revenue in April 2017"
result = chain.invoke(question)

print("=== SQL Output ===")
print(result['result'])


=== SQL Output ===
SELECT category, SUM(amount) AS total_revenue
FROM transaction
WHERE status = 'completed'
  AND date >= '2017-04-01' AND date <= '2017-04-30'
GROUP BY category
ORDER BY total_revenue DESC
LIMIT 5;


In [34]:
# === Run
question = "Identify the top 3 revenue sources that generated the highest revenue in January 2017."
result = chain.run(question)

print("=== SQL Output ===")
print(result)

=== SQL Output ===
SELECT category, SUM(amount) AS total_revenue
FROM transaction
WHERE status = 'completed'
  AND date BETWEEN '2017-01-01' AND '2017-01-31'
GROUP BY category
ORDER BY total_revenue DESC
LIMIT 3;
